In [105]:
import tensorflow as tf
import numpy as np
import jax
import jax.numpy as jnp
import jax.experimental.optimizers as optimizers
from jax.experimental import stax
from jax.experimental.stax import (Conv, Dense, MaxPool, Relu, Flatten)
from jax import jit, grad, random,vmap,value_and_grad
import jax.nn as jnn
from jax.tree_util import tree_multimap
import math
from scipy.special import softmax
import matplotlib.pyplot as plt
import warnings
from functools import partial # for use with vmap


In [2]:
def data_parse(record):
    features = {
        'N': tf.io.FixedLenFeature([], tf.int64),
        'labels': tf.io.FixedLenFeature([16], tf.float32),
        'elements': tf.io.VarLenFeature(tf.int64),
        'coords': tf.io.VarLenFeature(tf.float32),
    }
    parsed_features = tf.io.parse_single_example(
        serialized=record, features=features)
    coords = tf.reshape(tf.sparse.to_dense(parsed_features['coords'], default_value=0),[-1,4])
    elements = tf.sparse.to_dense(parsed_features['elements'], default_value=0)
    return (elements, coords), parsed_features['labels']
data = tf.data.TFRecordDataset(
    'qm9.tfrecords', compression_type='GZIP').map(data_parse)

[6 1 1 1 1] [[-1.2698136e-02  1.0858041e+00  8.0009960e-03 -5.3568900e-01]
 [ 2.1504159e-03 -6.0313176e-03  1.9761203e-03  1.3392100e-01]
 [ 1.0117308e+00  1.4637512e+00  2.7657481e-04  1.3392200e-01]
 [-5.4081506e-01  1.4475266e+00 -8.7664372e-01  1.3392299e-01]
 [-5.2381361e-01  1.4379326e+00  9.0639728e-01  1.3392299e-01]] [ 1.0000000e+00  1.5771181e+02  1.5770998e+02  1.5770699e+02
  0.0000000e+00  1.3210000e+01 -3.8769999e-01  1.1710000e-01
  5.0480002e-01  3.5364101e+01  4.4748999e-02 -4.0478931e+01
 -4.0476063e+01 -4.0475117e+01 -4.0498596e+01  6.4689999e+00]


In [3]:
#{'C':6,'H':1,'O':8,'N':7,'F':9}

def make_graph(e,x):
    e = e.numpy()
    x = x.numpy()
    r = x[:,:3]
    r2 = np.sum((r - r[:,np.newaxis,:])**2,axis=-1)
    edges = np.where(r2!=0, 1/r2,0.0) #[N,N]
    nodes = np.zeros((len(e),9))
    nodes[np.arange(len(e)), e-1] = 1
    return nodes,edges

def get_label(y):
    return y.numpy()[13]

In [4]:
for d in data:
    (e,x),y = d
    nodes,edges = make_graph(e,x)
    label = get_label(y)
    print (nodes.shape,edges.shape,label)
    break

(5, 9) (5, 5) -40.475117


/scratch/gwellawa/.conda/htf2/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in true_divide
  


In [65]:
def gcn_layer(nodes,edges,train_weights):
    
    #wq has shape (9,10), output from query has shape (5,10)
    query = jnp.dot(nodes,train_weights[0]) 
    

    #pairwise distances are used here wk has shape of (9,10), output from keys has shape (5,5,10)
    keys = jnp.dot(jnp.repeat(nodes[jnp.newaxis,...],nodes.shape[0],axis=0), train_weights[1]) * edges
    d_sq = math.sqrt(keys.shape[-1])
    b = jnn.softmax(query[jnp.newaxis,...] * keys/d_sq)
    
    #wv has shape (9,10), output from values has shape (5,5,10)
    values = jnp.dot(jnp.repeat(nodes[jnp.newaxis,...],nodes.shape[0],axis=0), train_weights[2])
    
    messages = b * values #out shape (5,5,10)
    
    net_message = jnp.mean(messages,axis= 1)
    self_message = nodes @ train_weights[3]
    
    #self loop
    out_nodes = jnn.relu((net_message+self_message)) 
    
    return out_nodes,edges
    
    
def graph_level_fts(nodes):
    node_avg = jnp.mean(nodes,axis=1)
    return node_avg

In [120]:
def loss(params, inputs, targets):
    # Computes average loss for the batch
    predictions = net_apply(params, inputs)
    
    #todo: add reg
    return np.mean((targets - predictions)**2)

@jit
def update(params, x, y, opt_state):
    """ Compute the gradient for a batch and update the parameters """
    value, grads = value_and_grad(loss)(params, x, y)
    opt_state = opt_update(0, grads, opt_state)
    return get_params(opt_state), opt_state, value

In [121]:
out_dim = 10
embed_dim = 4

#get node embeddings instead of one-hot 
element_embeddings = np.random.normal(size=(9,embed_dim))
embed_nodes = nodes @ element_embeddings

#get edge embeddings from pairwise distances
edge_embeddings = np.random.normal(size=(1,len(edges),out_dim))
embed_edges = edges[...,np.newaxis] * edge_embeddings

#trainable weights
w1 = np.random.normal(size = (4,embed_dim,out_dim))
w2 = np.random.normal(size = (4,embed_dim,out_dim))


#call gcn

n,e = gcn_layer(embed_nodes,embed_edges,w1)
n,e = gcn_layer(embed_nodes,embed_edges,w2)
n = graph_level_fts(n)
print(n,n.shape)

net_init, net_apply = stax.serial(
    Dense(16), Relu,
    Dense(1)
)

rng = random.PRNGKey(0)
in_shape = (-1,n.shape[0])

out_shape, net_params =net_init(rng,in_shape)
#print(net_params)

losses = loss(net_params,n,label)
#losses = vmap(partial(loss, net_params))(n, label)
#print(predictions)
print(losses,pred,label)

opt_init, opt_update,get_params = optimizers.adam(step_size=1e-2)
opt_state = opt_init(net_params)

params = get_params(opt_state)

up_params, opt_state, loss = update(params, n, label, opt_state)


[0.23964253 1.9595194  1.9574305  1.9660587  1.9568462 ] (5,)
1604.9153 [0.18368527] -40.475117


In [119]:
r = np.repeat(nodes[np.newaxis,...],nodes.shape[0],axis=0) @ np.ones((9,10 )) 
r.shape

a = jnp.ones((5,5))
b = jnp.ones((5,10))
c = a@b

print(c.shape)
#print(softmax(c).shape)

(5, 10)


In [ ]:
predictions = vmap(partial(net_apply, net_params))(xrange_inputs)
losses = vmap(partial(loss, net_params))(xrange_inputs, targets) # per-input loss
plt.plot(xrange_inputs, predictions, label='prediction')
plt.plot(xrange_inputs, losses, label='loss')
plt.plot(xrange_inputs, targets, label='target')
plt.legend()